In [ ]:
import json
import cv2
from tqdm import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
# Process annotations data
d_path_annot = '../MFD_datasets/annotations/MFD-balanced_test.json'
d_path_images = '../MFD_datasets/images'

processed_data = {
    'Id': [],
    'image_path': [],
    'semantic_masks': [],
}

with open(d_path_annot, 'r', encoding="utf-8") as json_file:
    json_data_dir = json.load(json_file)
    # Process image data
    for image_inf in tqdm(json_data_dir['images'], desc="Process images: "):
        real_img_id = image_inf['id']
        for k in processed_data:
            processed_data[k].append([])
        processed_data['Id'][-1] = real_img_id
        img_path = os.path.join(
            d_path_images, image_inf['file_name']
        )
        processed_data['image_path'][-1] = str(img_path)
        SIZE = (image_inf['height'], image_inf['width'], 3)
        processed_data['semantic_masks'][-1] = np.zeros(SIZE[:2], dtype=np.uint8)

with open(d_path_annot, 'r') as json_file:
    json_data_dir = json.load(json_file)
    image_id_old = ''
    skipped_counter = 1
    morph_kernel = cv2.getStructuringElement(
        cv2.MORPH_ELLIPSE, (3, 3)
    )

    for annotation_data in tqdm(
        json_data_dir['annotations'], desc="Process annotations: "
    ):
        # Data may be numerated to image of have through numeration
        process_image_id = annotation_data['image_id']

        image_data_indx = processed_data['Id'].index(process_image_id)
        label = annotation_data['category_id']
        # Process each granule
        for point_i, point in enumerate(annotation_data['segmentation']):
            if isinstance(point, list):
                point_xy = [
                    [point[j], point[j + 1]] for j in
                    range(0, len(point), 2)
                ]
                cnt = np.array(point_xy).reshape((-1, 1, 2)).astype(
                    np.int32
                )
                if len(cnt) < 3:  # bad contour
                    print('Cnt is bad')
                    continue

                single_mask = np.zeros((480, 480), dtype=np.uint8)
                _ = cv2.drawContours(
                    single_mask,
                    [cnt], -1, label, cv2.FILLED
                )
                single_mask = cv2.morphologyEx(
                    single_mask, cv2.MORPH_ERODE, morph_kernel, iterations=3
                )
                processed_data['semantic_masks'][image_data_indx][single_mask != 0] = label
                x, y, w, h = annotation_data["bbox"]
            else:
                continue

In [ ]:
image_indx = 0
test_image = cv2.imread(processed_data['image_path'][image_indx])
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

plt.rcParams['figure.figsize'] = [10, 10]
f, axarr = plt.subplots(1,2)
_ = axarr[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
_ = axarr[1].imshow(processed_data['semantic_masks'][image_indx], cmap='gray', vmin=0, vmax=1)